# NLP Basic Assignment
## NLP 맛보기 - spam.csv를 가지고 유의미한 해석을 도출해주세요!

In [6]:
# 필요한 모듈 불러오기
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from gensim.models import Word2Vec, FastText

## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다. 물론 7주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고, 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다 :)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
spam = pd.read_csv('/content/drive/MyDrive/dataset/spam.csv')

In [5]:
spam.iloc[5]['v2']

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [20]:
spam.v1 = (spam.v1 == 'spam')*1
spam

,v1,v2
0,0,"[go, jurong, point, crazy, available, bugis, n..."
1,0,"[ok, lar, joking, wif, u, oni]"
2,1,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,0,"[u, dun, say, early, hor, u, c, already, say]"
4,0,"[nah, think, goes, usf, lives, around, though]"
...,...,...
5567,1,"[nd, time, tried, contact, u, u, pound, prize,..."
5568,0,"[b, going, esplanade, fr, home]"
5569,0,"[pity, mood, suggestions]"
5570,0,"[guy, bitching, acted, like, interested, buyin..."


## Tokenizing


In [7]:
import nltk

In [13]:
spam.drop_duplicates(subset = ['v2'], inplace = True)
spam.v2.nunique()

5169

In [14]:
import re

def re_text(corpus):
    new = re.sub('[^a-zA-Z]', ' ', corpus).lower() 
    return new

spam.v2 = spam.v2.apply(re_text)

In [15]:
nltk.download('punkt')
nltk.download('stopwords') 
stops = list(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
tokens = []
for text in spam.v2 :            
    words = word_tokenize(text)   
    words = [word for word in words if word not in stops] 
    tokens.append(words)
spam.v2 = tokens    

In [17]:
spam[:5]

,v1,v2
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, think, goes, usf, lives, around, though]"


## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram, GloVe, FastText가 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

In [21]:
cnt_spam = CountVectorizer(max_features= 300, stop_words = 'english', lowercase= False)
cnt_ham = CountVectorizer(max_features= 300, stop_words = 'english', lowercase= False)

corpus_spam = sum(spam[spam.v1 == 1]['v2'], [])
corpus_ham = sum(spam[spam.v1 == 0]['v2'], [])

cnt_spam_t = cnt_spam.fit_transform(corpus_spam)
cnt_ham_t = cnt_ham.fit_transform(corpus_ham)

In [23]:
word_count_spam = pd.DataFrame( {'words' : cnt_spam.get_feature_names(),'counts' : cnt_spam_t.sum(axis = 0).flat} )
word_count_ham = pd.DataFrame( {'words' : cnt_ham.get_feature_names(),'counts' : cnt_ham_t.sum(axis = 0).flat} )

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
spam_de = word_count_spam.sort_values('counts', ascending = False)
ham_de = word_count_ham.sort_values('counts', ascending = False)
dupli = set()
i = 1
while True:
    set_spam = set(spam_de.head(i).words.tolist())
    set_ham = set(ham_de.head(i).words.tolist())
    common = set_ham.intersection(set_spam)
    if len(common) == 20: break
    if dupli != common: print( f"{i} 번째 중복 단어 : ",common - dupli)
    i += 1
    dupli = common.copy()

9 번째 중복 단어 :  {'ur'}
25 번째 중복 단어 :  {'send'}
40 번째 중복 단어 :  {'want'}
55 번째 중복 단어 :  {'phone'}
57 번째 중복 단어 :  {'text'}
61 번째 중복 단어 :  {'day'}
62 번째 중복 단어 :  {'number'}
64 번째 중복 단어 :  {'week'}
66 번째 중복 단어 :  {'new'}
76 번째 중복 단어 :  {'free'}
84 번째 중복 단어 :  {'know'}
85 번째 중복 단어 :  {'today'}
89 번째 중복 단어 :  {'time'}
91 번째 중복 단어 :  {'msg', 'yes'}
96 번째 중복 단어 :  {'message'}
105 번째 중복 단어 :  {'dear'}
119 번째 중복 단어 :  {'thanks'}
121 번째 중복 단어 :  {'hi'}
